In [27]:
import torch
import torch.nn as nn

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [67]:
from vit_pytorch import ViT, MAE
from vit_pytorch.slide_mae import SlideMAE

In [5]:
v = ViT(
    image_size = 256,
    patch_size = 32,
    num_classes = 1000,
    dim = 1024,
    depth = 6,
    heads = 8,
    mlp_dim = 2048
)

In [23]:
num_patches, encoder_dim = v.pos_embedding.shape[-2:]

In [24]:
num_patches

65

In [25]:
encoder_dim

1024

In [65]:
v.to_patch_embedding[2].weight.shape

torch.Size([1024, 3072])

In [20]:
# v.to_patch_embedding[2].weight.shape[-1]

In [28]:
to_patch = v.to_patch_embedding[0]
patch_to_emb = nn.Sequential(*v.to_patch_embedding[1:])

pixel_values_per_patch = v.to_patch_embedding[2].weight.shape[-1]

In [31]:
imgs = torch.randn(8, 3, 256, 256)

In [32]:
patches = to_patch(imgs)
batch, num_patches, *_ = patches.shape

In [33]:
patches.shape

torch.Size([8, 64, 3072])

In [34]:
batch, num_patches

(8, 64)

In [35]:
tokens = patch_to_emb(patches)
tokens.shape

torch.Size([8, 64, 1024])

In [36]:
v.pool

'cls'

In [37]:
# first patch is cls token
tokens += v.pos_embedding[:, 1:(num_patches + 1)]

In [38]:
tokens.shape

torch.Size([8, 64, 1024])

In [39]:
masking_ratio = .75
num_masked = int(masking_ratio * num_patches)
rand_indices = torch.rand(batch, num_patches).argsort(dim = -1)
masked_indices, unmasked_indices = rand_indices[:, :num_masked], rand_indices[:, num_masked:]

In [40]:
num_masked

48

In [42]:
rand_indices.shape

torch.Size([8, 64])

In [44]:
masked_indices.shape, unmasked_indices.shape

(torch.Size([8, 48]), torch.Size([8, 16]))

In [45]:
batch_range = torch.arange(batch)[:, None]
tokens = tokens[batch_range, unmasked_indices]

In [46]:
batch_range

tensor([[0],
        [1],
        [2],
        [3],
        [4],
        [5],
        [6],
        [7]])

In [51]:
torch.arange(batch).unsqueeze(-1)

tensor([[0],
        [1],
        [2],
        [3],
        [4],
        [5],
        [6],
        [7]])

In [53]:
batch_range.shape, unmasked_indices.shape

(torch.Size([8, 1]), torch.Size([8, 16]))

In [52]:
tokens.shape

torch.Size([8, 16, 1024])

In [59]:
masked_patches = patches[batch_range, masked_indices]
masked_patches.shape

torch.Size([8, 48, 3072])

In [60]:
encoded_tokens = v.transformer(tokens)

In [61]:
encoded_tokens.shape

torch.Size([8, 16, 1024])

In [63]:
v.transformer(all_tokens).shape

torch.Size([8, 64, 1024])

In [62]:
all_tokens = patch_to_emb(patches)
all_tokens.shape

torch.Size([8, 64, 1024])

In [68]:
v = ViT(
    image_size = 256,
    patch_size = 32,
    num_classes = 1000,
    dim = 1024,
    depth = 6,
    heads = 8,
    mlp_dim = 2048
)

In [70]:
mae = SlideMAE(
    encoder = v,
    n_slides = 10,
    masking_ratio = 0.75,   # the paper recommended 75% masked patches
    decoder_dim = 512,      # paper showed good results with just 512
    decoder_depth = 6       # anywhere from 1 to 8
)


In [71]:
images = torch.randn(8, 3, 256, 256)

In [73]:
slides = torch.randint(0, 10, (8,))

In [78]:
loss = mae(images, slides)

In [79]:
loss

tensor(1.8420, grad_fn=<MseLossBackward0>)

In [80]:
loss.backward()